In [1]:
from robotic import ry
import numpy as np
import time

###########################################PART-2 STARTS HERE##########################################

In [2]:
C = ry.Config()
C.addFile('cargobot.g')
C.addFile('maze.g')
C.addFile('cargo.g')

qHome = C.getJointState()
C.setJointState(qHome)

Get Final Pose with KOMO

In [3]:
S = ry.Skeleton()

S.addEntry([1., 1.], ry.SY.touch,  ["l_gripper", "cargo_handle"])
#S.addEntry([1., 2.], ry.SY.stable,  ["l_panda_finger_joint1", "cargo_handle"])

#S.addEntry([1., 7.], ry.SY.stable,  ["l_gripper", "l_panda_finger_joint2"])
S.addEntry([1., 7.], ry.SY.stable,  ["l_gripper", "cargo_handle"])
#S.addEntry([0.8, 1.2], ry.SY.downUp, ["l_panda_finger_joint1"])
#S.addEntry([1.8, 2.2], ry.SY.downUp, ["l_panda_finger_joint1"])

S.addEntry([2., 2.], ry.SY.poseEq,  ["cargo", "cp1"])
#S.addEntry([2., -1.], ry.SY.stable,  ["cp1", "cargo"])

S.addEntry([3., 3.], ry.SY.poseEq,  ["cargo", "cp2"])
#S.addEntry([3., -1.], ry.SY.stable,  ["cp2", "cargo"])

S.addEntry([4., 4.], ry.SY.poseEq,  ["cargo", "cp3"])
#S.addEntry([4., -1.], ry.SY.stable,  ["cp3", "cargo"])

S.addEntry([5., 5.], ry.SY.poseEq,  ["cargo", "cp4"])
#S.addEntry([5., -1.], ry.SY.stable,  ["cp4", "cargo"])

S.addEntry([6., 6.], ry.SY.poseEq,  ["cargo", "cp5"])
#S.addEntry([6., -1.], ry.SY.stable,  ["cp5", "cargo"])

S.addEntry([7., 7.], ry.SY.poseEq,  ["cargo", "final"])
#S.addEntry([7., -1.], ry.SY.stable,  ["final", "cargo"])

S.enableAccumulatedCollisions(True)

komo = S.getKomo_waypoints(C, 1e-1, 1e-2,0)
nlp = komo.nlp()
sol = ry.NLP_Solver()
sol.setProblem(nlp)
sol.setOptions( stopTolerance=1e-2 )
ret = sol.solve()
waypoints = komo.getPath_qAll()
#waypoints = komo.getPath()[0]
komo.view_play(True, .2)
#C.setJointState(q[q.shape[0]-1])

====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
==nlp== it:   0  evals:  66  A(x):   0.496802  f:   0.353638  g:          0  h:    1.12749  |x-x'|:    4.18488 	stop:DeltaConverge
==nlp== it:   1  evals:  66  A(x):    1.35579  mu:5
==nlp== it:   1  evals:  89  A(x):    1.25665  f:   0.353899  g:          0  h:    1.06191  |x-x'|: 0.00863185 	stop:TinyXSteps (bad:1)
==nlp== StoppingCriterion Delta<0.01


1

RRT

In [4]:
#define a path finding problem
m = len(waypoints)
rrt_dofs = []
rrt_paths = []
for i in range(0, int(m)):
    [Ctmp, q0, q1] = S.getTwoWaypointProblem(i, komo)
    Ctmp.setJointState(q0)
    Ctmp.setJointState(q1)
    rrt = ry.PathFinder()
    rrt.setProblem(Ctmp, q0, q1)
    ret = rrt.solve()
    path = ret.x
    #rrt_paths.append(rrt)
    #rrt_dofs.append(Ctmp.getDofIDs())
    print("rettttt: ", ret)
    #print("path size:", path)
    # display the path
    if hasattr(ret, 'x') and isinstance(ret.x, np.ndarray) and ret.x.ndim == 2 and ret.x.shape[0] > 0:
        rrt_paths.append(ret.x)
        rrt_dofs.append(Ctmp.getDofIDs())
        for t in range(0, path.shape[0]):
            Ctmp.setJointState(path[t])
            Ctmp.view(False)
            time.sleep(0.02)
    

rettttt:  { time: 0.015741, evals: 49, done: 1, feasible: 1, sos: -1, f: -1, ineq: -1, eq: -1 }
rettttt:  { time: 2.35803, evals: 3194, done: 1, feasible: 1, sos: -1, f: -1, ineq: -1, eq: -1 }
rettttt:  { time: 0.05426, evals: 87, done: 1, feasible: 1, sos: -1, f: -1, ineq: -1, eq: -1 }
rettttt:  { time: 0.304987, evals: 572, done: 1, feasible: 1, sos: -1, f: -1, ineq: -1, eq: -1 }
rettttt:  { time: 1.38345, evals: 2277, done: 1, feasible: 1, sos: -1, f: -1, ineq: -1, eq: -1 }
rettttt:  { time: 0.949299, evals: 1351, done: 1, feasible: 1, sos: -1, f: -1, ineq: -1, eq: -1 }
rettttt:  { time: 0.883668, evals: 1304, done: 1, feasible: 1, sos: -1, f: -1, ineq: -1, eq: -1 }


In [5]:
C.setJointState(qHome)
komo2 = S.getKomo_path(C, 60, 1e0, 1e0, 1e-2, 1e0)
komo2.initWithWaypoints(waypoints)


[59, 119, 179, 239, 299, 359, 419]

In [6]:

for t in range(0, int(m)):
    komo2.initPhaseWithDofsPath(t, rrt_dofs[t], rrt_paths[t], True)

ret = ry.NLP_Solver(komo2.nlp(), verbose=0).solve()
q = komo2.getPath()
print('last path:', q)

# display the path for part 1
for i in range(0, q.shape[0]):
    C.setJointState(q[i])
    C.view(False, f'waypoints{i}')
    time.sleep(0.02)

last path: [[ 8.41846612e-04 -8.01275316e-04  1.58543399e-03 ...  2.45627628e-03
   1.99990559e+00 -5.01316175e-01]
 [ 2.50101597e-03 -2.39742803e-03  4.68527868e-03 ...  7.26814288e-03
   1.99970269e+00 -5.03890354e-01]
 [ 4.95297886e-03 -4.78205445e-03  9.22851699e-03 ...  1.43349193e-02
   1.99937727e+00 -5.07664371e-01]
 ...
 [-3.91575982e+00 -2.47312968e+00  1.03037593e+00 ...  9.25097408e-01
   1.94015048e+00  5.52963124e-01]
 [-4.01817688e+00 -2.47684431e+00  1.05472484e+00 ...  9.60869383e-01
   1.96207701e+00  5.97028837e-01]
 [-4.12098325e+00 -2.48071933e+00  1.07908436e+00 ...  9.96655450e-01
   1.98399134e+00  6.41164865e-01]]
